<a href="https://colab.research.google.com/github/YSKIM-moya/AIFFEL_quest_rs/blob/master/MainQuest/Quest04/miniaffel_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install U -q langchain

In [ ]:
!pip install -U -q langchain-google-genai

In [ ]:
!pip install U -q langchain-community langchain-core

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 6.6 MB/s eta 

In [ ]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.3/754.3 kB 9.0 MB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv
import os

# 파일 경로에 맞게 지정 (Colab 현재 디렉토리 기준)
load_dotenv("/content/secret.env")

# 환경변수에서 API 키 읽기
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")
# print(api_key)  # 확인용, 실제 코드에선 출력 안하는 것이 안전

In [ ]:
import pandas as pd
import torch
import numpy as np
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import JSONLoader
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import RetrievalQA

import glob
import json
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/
#!unzip /content/drive/MyDrive/VL_06.중학교_1학년_03.수학_01.텍스트.zip -d /content/drive/MyDrive/mini_aiffel/json_me

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
json_list = glob.glob('/content/drive/MyDrive/mini_aiffel/json_me/TL_06.중학교 1학년_03.수학_01.텍스트/*.json') # json_list 가져오기
len(json_list)

9124

In [ ]:
# json 파일 예시
json_data_example = {
    "raw_data_info": {
        "raw_data_name": "수학_중학교_3학년_text_1153_png_17",
        "date": "2024-11-28",
        "publisher": "2차 저작",
        "publication_year": "2024-11-06",
        "school": "중학교",
        "grade": "3학년",
        "semester": "2학기",
        "subject": "수학",
        "revision_year": "2015"
    },
    "source_data_info": {
        "source_data_name": "S1_중등_3_수학_TXT_3102614",
        "2009_achievement_standard": [
            " "
        ],
        "2015_achievement_standard": [
            "[9수05-07] 분산과 표준편차의 의미를 이해하고, 이를 구할 수 있다."
        ],
        "2022_achievement_standard": [
            "[9수05-07] 분산과 표준편차의 의미를 이해하고, 이를 구할 수 있다."
        ]
    },
    "learning_data_info": {
        "learning_data_name": "S1_중등_3_수학_TXT_3102614",
        "class_num": 5,
        "class_name": "텍스트",
        "bounding_box": [
            [
                0,
                0,
                297,
                24
            ]
        ],
        "text_description": "제곱한 값의 평균을 구하여 분산을 구합니다.",
        "text_qa": "제곱한 값의 평균을 구하는 방법은 무엇인가요?",
        "text_an": "제곱한 값의 평균을 구하려면 먼저 각 데이터의 값을 제곱한 다음, 그 제곱한 값들의 총합을 구하고, 그 총합을 데이터의 개수로 나눕니다."
    }
}

In [ ]:
import uuid # 고유 ID 생성을 위해 추가

def create_document_from_json(file_name, data):
    # 고유 ID 생성 (raw_data_name이 유일하지 않을 경우 대비)
    #doc_id = data.get("raw_data_info", {}).get("raw_data_name", str(uuid.uuid4()))
    doc_id = file_name

    # content 구성 (핵심 정보 + 관련 텍스트)
    subject = data.get("raw_data_info", {}).get("subject", "정보 없음")
    grade = data.get("raw_data_info", {}).get("grade", "정보 없음")

    # 2015_achievement_standard는 리스트이므로 첫 번째 요소를 가져오거나, 모든 요소를 결합
    achievement_standard_2015 = data.get("source_data_info", {}).get("2015_achievement_standard", [])
    standard_text = achievement_standard_2015[0] if achievement_standard_2015 else "정보 없음"

    text_description = data.get("learning_data_info", {}).get("text_description", "")
    text_qa = data.get("learning_data_info", {}).get("text_qa", "")
    text_an = data.get("learning_data_info", {}).get("text_an", "")

    # Content에 모든 관련 텍스트를 논리적인 흐름으로 결합
    # 여기에 어떤 순서로, 어떤 레이블과 함께 넣을지 결정합니다.
    content_parts = []

    # 본문 내용 추가
    if standard_text != "정보 없음":
        content_parts.append(f"학습 목표: {standard_text}")
    if text_qa:
        content_parts.append(f"질문: {text_qa}")
    if text_an:
        content_parts.append(f"답변: {text_an}")
    if text_description:
        content_parts.append(f"설명: {text_description}")



    combined_content = "\n\n".join(content_parts) # 각 부분을 두 줄씩 띄워서 구분

    # 메타데이터 구성 (필터링 및 추가 정보 제공용)
    metadata = {
        "id": doc_id, # 고유 식별자 (이곳에 저장됩니다!)
        "grade": data.get("raw_data_info", {}).get("grade"),
        "subject": data.get("raw_data_info", {}).get("subject"),
        "LearningObjective" : standard_text,
        "raw_data_name": data.get("raw_data_info", {}).get("raw_data_name"),
        "date": data.get("raw_data_info", {}).get("date"),
    }

    return Document(page_content=combined_content, metadata=metadata)


In [ ]:
# 변환 함수 사용
document_instance = create_document_from_json("test_file", json_data_example)

print("--- 생성된 Document의 page_content ---")
# 변경된 부분: doc_id는 Document 객체의 직접적인 속성이 아니라 metadata 안에 있습니다.

print(document_instance.page_content)
print("\n--- 생성된 Document의 metadata (일부) ---")
print(f"ID: {document_instance.metadata.get('id')}")
print(f"LearningObjective: {document_instance.metadata.get('LearningObjective')}")


--- 생성된 Document의 page_content ---
과목: 수학

학년: 3학년

학습 목표: [9수05-07] 분산과 표준편차의 의미를 이해하고, 이를 구할 수 있다.

설명: 제곱한 값의 평균을 구하여 분산을 구합니다.

질문: 제곱한 값의 평균을 구하는 방법은 무엇인가요?

답변: 제곱한 값의 평균을 구하려면 먼저 각 데이터의 값을 제곱한 다음, 그 제곱한 값들의 총합을 구하고, 그 총합을 데이터의 개수로 나눕니다.

--- 생성된 Document의 metadata (일부) ---
ID: test_file
LearningObjective: [9수05-07] 분산과 표준편차의 의미를 이해하고, 이를 구할 수 있다.


In [ ]:
processed_files_count = 0
all_docs = []
print(f"\n--- 모든 JSON 파일 처리 시작 ---")

for file_path in json_list:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        # create_document_from_json 함수를 사용하여 Document 객체 생성
        file_name = os.path.basename(file_path)   # 'S1_중등_1_수학_TXT_3017092.json'
        file_id = os.path.splitext(file_name)[0]  # 'S1_중등_1_수학_TXT_3017092'
        doc = create_document_from_json(file_id, data)
        all_docs.append(doc)
        processed_files_count += 1

    except json.JSONDecodeError:
        print(f"⚠️ 경고: {file_path} 파일이 유효한 JSON 형식이 아닙니다. 건너뜁니다.")
        continue
    except FileNotFoundError:
        print(f"⚠️ 경고: {file_path} 파일을 찾을 수 없습니다. 건너뜁니다.")
        continue
    except Exception as e:
        print(f"❌ 오류: {file_path} 파일 처리 중 예상치 못한 오류 발생: {e}. 건너킵니다.")
        continue

print(f"✅ 총 {processed_files_count}개의 유효한 JSON 파일이 문서로 변환되었습니다.")


--- 모든 JSON 파일 처리 시작 ---
✅ 총 9124개의 유효한 JSON 파일이 문서로 변환되었습니다.


In [ ]:
for file_path in json_list:
        # create_document_from_json 함수를 사용하여 Document 객체 생성
        file_name = os.path.basename(file_path)
        id_name = os.path.splitext(file_name)[0]  # 'S1_중등_1_수학_TXT_3017092'
        print(id_name)
        break

S1_중등_1_수학_TXT_3015626


In [ ]:
print(all_docs[10])

page_content='학습 목표: [9수04-03] 삼각형을 작도할 수 있다.

질문: 직각 삼각형이란 무엇인가요?

답변: 직각 삼각형은 하나의 각이 90도인 삼각형입니다.

설명: 직각 삼각형: 하나의 각이 90도인 삼각형' metadata={'id': 'S1_중등_1_수학_TXT_3017092', 'grade': '1학년', 'subject': '수학', 'LearningObjective': '[9수04-03] 삼각형을 작도할 수 있다.', 'raw_data_name': '수학_중학교_1학년_text_3226_15_png_04', 'date': '2024-11-20'}


In [ ]:
print(all_docs[100])

page_content='학습 목표: [9수04-01] 점, 선, 면, 각을 이해하고, 점, 직선, 평면의 위치 관계를 설명할 수 있다.

질문: 종이에 표시한 작은 점은 무엇을 의미할까요?

답변: 작은 점은 점이나 위치를 나타낼 수 있으며, 수학에서는 coordinate plane에서 점의 위치를 표시할 때 사용됩니다.

설명: 예를 들어, 종이에 표시한 작은 점을 생각해 보세요.' metadata={'id': 'S1_중등_1_수학_TXT_3091688', 'grade': '1학년', 'subject': '수학', 'LearningObjective': '[9수04-01] 점, 선, 면, 각을 이해하고, 점, 직선, 평면의 위치 관계를 설명할 수 있다.', 'raw_data_name': '수학_중학교_1학년_text_483_png_01', 'date': '2024-11-28'}


Embedding Model
- genai 라이브러리의 list_models 함수를 사용하여 사용 가능한 모델들의 목록을 가져옵니다.

In [ ]:
import google.generativeai as genai
for model in genai.list_models():
    if "embedContent" in model.supported_generation_methods:
        print(model.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
#embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-exp-03-07")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

Step4 : VectorStore 사용해보기

VectorStore는 자연어를 Vector로 embedding한 후, 이를 저장하는 공간입니다.

쿼리나 문서 등의 Emabedding이 저장된 후, 재사용 및 참조를 위해 VectorStroe에서 빠르게 유사도를 측정, 탐색할 수 있습니다.
VectorStore로는 Chroma나 Faiss를 가장 많이 사용합니다.

In [ ]:
persist_directory = './my_vector_db_2'


os.makedirs(persist_directory, exist_ok=True)

vectorstore = None

In [ ]:
from langchain_community.vectorstores import Chroma

if all_docs: # 변환된 문서가 있다면
    print(f"\n--- 변환된 문서({len(all_docs)}개) 바로 벡터 스토어에 추가 중... ---")

    # ✅ 청크 분할 적용
    chunked_docs = text_splitter.split_documents(all_docs)

    print(f"📎 문서를 총 {len(chunked_docs)}개의 청크로 분할 완료!")

    # ✅ Chroma 벡터 스토어에 저장
    vectorstore = Chroma.from_documents(
        #documents=all_docs,
        documents=chunked_docs,
        embedding=embedding_model,
        persist_directory=persist_directory
    )
    print(f"✨ 벡터 스토어에 총 {len(chunked_docs)}개의 청크가 저장되었습니다.")
    #print(f"✨ 모든 JSON 파일 처리 완료! 최종적으로 벡터 스토어에 {len(all_docs)}개의 문서가 추가되었습니다.")
else:
    print(f"\n➡️ 처리할 유효한 JSON 문서가 없어 벡터 스토어에 추가된 문서가 없습니다.")


--- 변환된 문서(9124개) 바로 벡터 스토어에 추가 중... ---
📎 문서를 총 9137개의 청크로 분할 완료!
✨ 벡터 스토어에 총 9137개의 청크가 저장되었습니다.


벡터 스토어 검색 테스트

In [ ]:
print("\n--- 벡터 스토어 검색 테스트 ---")
try:
    if vectorstore:

        query = "소인수분해는 무엇인가요?"
        docs_found = vectorstore.similarity_search(query, k=5)

        print(f"쿼리: '{query}'에 대한 검색 결과:")
        for i, doc in enumerate(docs_found):
            print(f"\n--- 문서 {i+1} ---")
            print(f"내용 (일부): {doc.page_content[:200]}...") # 원본 Document가 그대로 저장되므로 길이에 따라 잘릴 수 있음
            print(f"메타데이터 (과목/학년): {doc.metadata.get('subject')}, {doc.metadata.get('grade')}")
            print(f"메타데이터 (id): {doc.metadata.get('id')}")
    else:
        print("벡터 스토어에 문서가 추가되지 않아 검색을 수행할 수 없습니다.")

except Exception as e:
    print(f"벡터 스토어 검색 중 오류 발생: {e}")


--- 벡터 스토어 검색 테스트 ---
쿼리: '소인수분해는 무엇인가요?'에 대한 검색 결과:

--- 문서 1 ---
내용 (일부): 학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 무엇인가요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다....
메타데이터 (과목/학년): 수학, 1학년
메타데이터 (id): S1_중등_1_수학_TXT_160843

--- 문서 2 ---
내용 (일부): 학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 무엇인가요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다....
메타데이터 (과목/학년): 수학, 1학년
메타데이터 (id): S1_중등_1_수학_TXT_160843

--- 문서 3 ---
내용 (일부): 학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 무엇인가요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다....
메타데이터 (과목/학년): 수학, 1학년
메타데이터 (id): S1_중등_1_수학_TXT_160843

--- 문서 4 ---
내용 (일부): 학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 어떻게 되나요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다....
메타데이터 (과목/학년): 수학, 1학년
메타데이터 (id): S1_중등_1_수학_TXT_160840

--- 문서 5 ---
내용 (일부): 학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 어떻게 되나요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다....
메타데이터 (과목/학년): 수학, 1학년
메타데이터 (id): S1_중등_1_수학_TXT_160840


소인수분해가 포함된 문서를 확인하려면?

In [ ]:
cnt = 0
for doc in all_docs:
    if "소인수분해" in doc.page_content:
        cnt +=1
        if cnt < 10 :
          print(doc.page_content)

학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 소인수란 무엇인가요?

답변: 소인수는 1보다 큰 자연수 중에서 어떤 수를 나누었을 때 나누어 떨어지는 수를 의미합니다. 예를 들어, 12의 소인수는 2, 3가 있습니다.

설명: 소인수란, 1보다 큰 자연수 중에서 어떤 수를 나누었을 때 나누어떨어지는 수를 의미합니다.
학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 12를 인수분해하면 어떤 숫자들이 나오나요?

답변: 12는 1, 2, 3, 4, 6, 12로 인수분해할 수 있습니다. 예를 들어, 12는 3 x 4로도 나타낼 수 있습니다.

설명: 12는 다음과 같이 인수분해할 수 있습니다:
학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 15를 3으로 나누면 얼마인가요?

답변: $15 \div 3 = 5$입니다.

설명: 15는 3으로 나눌 수 있으므로, $15 \div 3 = 5$입니다.
학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 소인수분해란 무엇인가요?

답변: 소인수분해는 자연수를 소수(1보다 큰 자연수 중에서 자기가 아닌 자연수로 나눌 수 없는 수)의 곱으로 표현하는 방법입니다. 예를 들어, 12는 2 × 2 × 3으로 소인수분해할 수 있습니다.

설명: 자연수를 소인수분해하는 방법을 살펴보겠습니다.
학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 소수란 무엇인가요?

답변: 소수는 1과 자기 자신만으로 나눌 수 있는 1보다 큰 자연수입니다. 예를 들면 2, 3, 5, 7, 11이 소수입니다.

설명: 예를 들어, 2, 3, 5, 7, 11 등이 소수입니다.
학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 6을 2로 나누면 어떤 수가 나오나요?

답변: 6을 2

In [ ]:
keyword = "소인수분해"  # 또는 "순환소수"
matching_docs = []

# Chroma 벡터스토어 기준
if hasattr(vectorstore, "get"):
    stored_docs = vectorstore.get()["documents"]

    for i, doc in enumerate(stored_docs):
        if any(keyword in chunk for chunk in doc):
            print(f"✅ {i+1}번째 문서에서 키워드 '{keyword}' 발견")
            print(doc)
            matching_docs.append(doc)
else:
    print("⚠️ 이 벡터스토어에서는 직접 저장된 문서를 확인할 수 없습니다.")

In [ ]:
def search_keyword_from_vetcor(keyword):
    count = 0
    if hasattr(vectorstore, "get"):
        all_data = vectorstore.get()
        print("vectorstore.get() 결과 키:", all_data.keys())

        stored_docs = all_data.get("documents", None)
        if stored_docs is None:
            print("⚠️ 'documents' 키가 존재하지 않습니다. 대신 'metadatas' 또는 다른 키 확인:")
            print(all_data.keys())
        else:
            print(f"총 문서 수: {len(stored_docs)}")
            keyword = "소인수분해"
            matching_docs = []
            for i, doc in enumerate(stored_docs):
                # doc가 문자열이 아닐 수도 있으니 str로 변환
                if keyword in str(doc):
                    print(f"✅ {i+1}번째 문서에서 키워드 '{keyword}' 발견")
                    print(doc)
                    matching_docs.append(doc)
                    count +=1
                if count > 10 :
                   break
            if not matching_docs:
                print(f"키워드 '{keyword}'가 포함된 문서를 찾지 못했습니다.")
    else:
        print("⚠️ 이 벡터스토어에서는 'get' 메서드를 지원하지 않습니다.")


In [ ]:
search_keyword_from_vetcor("소인수분해")

vectorstore.get() 결과 키: dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])
총 문서 수: 9137
✅ 3번째 문서에서 키워드 '소인수분해' 발견
학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 소인수란 무엇인가요?

답변: 소인수는 1보다 큰 자연수 중에서 어떤 수를 나누었을 때 나누어 떨어지는 수를 의미합니다. 예를 들어, 12의 소인수는 2, 3가 있습니다.

설명: 소인수란, 1보다 큰 자연수 중에서 어떤 수를 나누었을 때 나누어떨어지는 수를 의미합니다.
✅ 10번째 문서에서 키워드 '소인수분해' 발견
학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 12를 인수분해하면 어떤 숫자들이 나오나요?

답변: 12는 1, 2, 3, 4, 6, 12로 인수분해할 수 있습니다. 예를 들어, 12는 3 x 4로도 나타낼 수 있습니다.

설명: 12는 다음과 같이 인수분해할 수 있습니다:
✅ 24번째 문서에서 키워드 '소인수분해' 발견
학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 15를 3으로 나누면 얼마인가요?

답변: $15 \div 3 = 5$입니다.

설명: 15는 3으로 나눌 수 있으므로, $15 \div 3 = 5$입니다.
✅ 83번째 문서에서 키워드 '소인수분해' 발견
학습 목표: [9수01-01] 소인수분해의 뜻을 알고, 자연수를 소인수분해할 수 있다.

질문: 소인수분해란 무엇인가요?

답변: 소인수분해는 자연수를 소수(1보다 큰 자연수 중에서 자기가 아닌 자연수로 나눌 수 없는 수)의 곱으로 표현하는 방법입니다. 예를 들어, 12는 2 × 2 × 3으로 소인수분해할 수 있습니다.

설명: 자연수를 소인수분해하는 방법을 살펴보겠습니다.
✅ 93번째 문서에서 키워드 '소인수분해' 발견
학

In [ ]:
keyword = "회전체"  # 또는 "순환소수"
matching_docs = []

# Chroma 벡터스토어 기준
if hasattr(vectorstore, "get"):
    stored_docs = vectorstore.get()["documents"]

    for i, doc in enumerate(stored_docs):
        if any(keyword in chunk for chunk in doc):
            print(f"✅ {i+1}번째 문서에서 키워드 '{keyword}' 발견")
            print(doc)
            matching_docs.append(doc)
else:
    print("⚠️ 이 벡터스토어에서는 직접 저장된 문서를 확인할 수 없습니다.")

In [ ]:
print(f"총 all_docs 문서 수: {len(all_docs)}")
vector_docs = vectorstore.get()["documents"]
print(f"총 vectorstore 저장 문서 수: {len(vector_docs)}")

총 all_docs 문서 수: 9124
총 vectorstore 저장 문서 수: 39320


In [ ]:
# 누락된 문서들만 filtered_docs 리스트로 구성
missing_docs = [doc for doc in all_docs if "소인수분해" in doc.page_content or "순환소수" in doc.page_content]

# 벡터스토어에 추가
vectorstore.add_documents(documents=missing_docs)
print(f"✅ 누락된 문서 {len(missing_docs)}개가 벡터스토어에 추가되었습니다.")

✅ 누락된 문서 485개가 벡터스토어에 추가되었습니다.


In [ ]:
print(f"총 all_docs 문서 수: {len(all_docs)}")
vector_docs = vectorstore.get()["documents"]
print(f"총 vectorstore 저장 문서 수: {len(vector_docs)}")

총 all_docs 문서 수: 9124
총 vectorstore 저장 문서 수: 39805


Step5 : Retriever 사용해보기

Retriever는 사용자 입력 시 임베딩 모델을 거친 쿼리와 vectorstore에 저장된 사전 정보들 간의 유사 문장을 찾아내어 출력하는 역할을 합니다.

긴 문서에서 원하는 답변을 찾을 수 있는 RetrieverQA 체인을 사용합니다

In [ ]:
#랭체인의 구글Api를 사용합니다
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from IPython.display import Markdown, display

# 	LangChain에서 Gemini Chat API를 사용하는 래퍼 클래스
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temperature=0.0)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 체인 타입 (stuff, map_reduce, refine, map_rerank)
    retriever=vectorstore.as_retriever(
        #search_type="mmr", # 검색 방식: MMR (Maximum Marginal Relevance)
        #search_kwargs={"k": 3, "fetch_k": 20} # 검색 인자: k=3 (최종 반환 문서 수), fetch_k=10 (MMR을 위한 초기 후보군)
        search_type="similarity",
        search_kwargs={"k": 10} # 검색 인자: k=3 (최종 반환 문서 수), fetch_k=10 (MMR을 위한 초기 후보군)
    ),
    return_source_documents=True # 원본 문서를 함께 반환할지 여부
)

1) Retrieval QA Only

In [ ]:
query = "소인수분해란 무엇인가요?"
result = qa(query)

import pprint
pprint.pprint(result)

display(Markdown(result["result"]))

retrieved_docs = result["source_documents"] # 검색된 문서들

for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Document {i+1} ---")
    print(doc.page_content[:1000])  # 처음 1000자만 출력
    print(f"Metadata: {doc.metadata}")

{'query': '소인수분해란 무엇인가요?',
 'result': '죄송합니다. 그 질문에 대한 답을 모르겠습니다.',
 'source_documents': [Document(metadata={'subject': '수학', 'raw_data_name': '수학_중학교_1학년_text_3226_35_png_06', 'grade': '1학년', 'LearningObjective': '[9수03-01] 순서쌍과 좌표를 이해한다.', 'date': '2024-11-20', 'id': 'S1_중등_1_수학_TXT_3010447'}, page_content='학습 목표: [9수03-01] 순서쌍과 좌표를 이해한다.\n\n질문: x는 무엇을 나타내나요?\n\n답변: x는 x축에서의 거리입니다.\n\n설명: x는 x축에서의 거리입니다.'),
                      Document(metadata={'id': 'S1_중등_1_수학_TXT_160843', 'subject': '수학', 'date': '2024-08-27', 'grade': '1학년', 'LearningObjective': '[9수04-08] 회전체의 성질을 이해한다.', 'raw_data_name': 'f3b7f237-1ce2-4225-8d5f-bd8bed877be0'}, page_content='학습 목표: [9수04-08] 회전체의 성질을 이해한다.\n\n질문:  단면의 모양이 무엇인가요?\n\n답변: 단면의 모양은 원입니다.\n\n설명: (2) 단면의 모양은 원이다.'),
                      Document(metadata={'date': '2024-08-27', 'raw_data_name': 'f2c3442b-6382-49d1-a3ab-86dbc0509c77', 'id': 'S1_중등_1_수학_TXT_160840', 'LearningObjective': '[9수04-08] 회전체의 성질을 이해한다.', 'subject': '수학'

죄송합니다. 그 질문에 대한 답을 모르겠습니다.


--- Document 1 ---
학습 목표: [9수03-01] 순서쌍과 좌표를 이해한다.

질문: x는 무엇을 나타내나요?

답변: x는 x축에서의 거리입니다.

설명: x는 x축에서의 거리입니다.
Metadata: {'subject': '수학', 'raw_data_name': '수학_중학교_1학년_text_3226_35_png_06', 'grade': '1학년', 'LearningObjective': '[9수03-01] 순서쌍과 좌표를 이해한다.', 'date': '2024-11-20', 'id': 'S1_중등_1_수학_TXT_3010447'}

--- Document 2 ---
학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 무엇인가요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다.
Metadata: {'id': 'S1_중등_1_수학_TXT_160843', 'subject': '수학', 'date': '2024-08-27', 'grade': '1학년', 'LearningObjective': '[9수04-08] 회전체의 성질을 이해한다.', 'raw_data_name': 'f3b7f237-1ce2-4225-8d5f-bd8bed877be0'}

--- Document 3 ---
학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 어떻게 되나요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다.
Metadata: {'date': '2024-08-27', 'raw_data_name': 'f2c3442b-6382-49d1-a3ab-86dbc0509c77', 'id': 'S1_중등_1_수학_TXT_160840', 'LearningObjective': '[9수04-08] 회전체의 성질을 이해한다.', 'subject': '수학', 'grade': '1학년'}

--- Document 4 ---
학습 목표: [9수04

In [ ]:
query = "회전체 대해 설명해줘"
result = qa(query)
display(Markdown(result["result"]))

pprint.pprint(result)

제공된 텍스트에는 회전체에 대한 자세한 설명이 없습니다.  회전체를 자르면 평면도형이 생긴다는 정보만 있습니다.  더 자세한 설명은 드릴 수 없습니다.

{'query': '회전체 대해 설명해줘',
 'result': '제공된 텍스트에는 회전체에 대한 자세한 설명이 없습니다.  회전체를 자르면 평면도형이 생긴다는 정보만 있습니다.  더 '
           '자세한 설명은 드릴 수 없습니다.',
 'source_documents': [Document(metadata={'grade': '1학년', 'LearningObjective': '[9수04-07] 다면체의 성질을 이해한다.', 'id': 'S1_중등_1_수학_TXT_3091829', 'subject': '수학', 'date': '2024-11-28', 'raw_data_name': '수학_중학교_1학년_text_514_png_17'}, page_content='학습 목표: [9수04-07] 다면체의 성질을 이해한다.\n\n질문: 오일러의 정리는 무엇인가요?'),
                      Document(metadata={'date': '2024-11-20', 'id': 'S1_중등_1_수학_TXT_3016340', 'raw_data_name': '수학_중학교_1학년_text_3226_110_png_17', 'grade': '1학년', 'LearningObjective': '[9수03-01] 순서쌍과 좌표를 이해한다.', 'subject': '수학'}, page_content='학습 목표: [9수03-01] 순서쌍과 좌표를 이해한다.\n\n질문: 두 위치를 연결하여 점을 찍는 것은 어떤 의미인가요?'),
                      Document(metadata={'raw_data_name': '527339e4-f9f2-43cb-a541-12424db280b4', 'grade': '1학년', 'date': '2024-08-27', 'subject': '수학', 'LearningObjective': '[9수04-08] 회전체의 성질을 이해한다.', 'id': 'S1_중등_1_수학_TXT_1034

2) LLM Only

In [ ]:
request = llm.invoke("회전체 대해 설명해줘")
display(Markdown(request.content))

회전체는 **평면도형을 어떤 직선을 축으로 하여 360도 회전시켜 만들어지는 입체도형**입니다.  회전하는 평면도형을 **모양도형** 또는 **생성도형**이라고 하고, 회전축을 **회전축**이라고 합니다.

회전체의 특징은 다음과 같습니다.

* **대칭성:** 회전축에 대해 회전대칭입니다.  즉, 회전축을 중심으로 어느 각도로 회전시켜도 모양이 변하지 않습니다.
* **곡면:** 일반적으로 곡면으로 이루어져 있습니다.  단, 모양도형이 직선이나 원 등으로 이루어진 경우 일부 평면이 포함될 수 있습니다.
* **부피와 겉넓이:** 모양도형의 형태와 회전축으로부터의 거리에 따라 부피와 겉넓이가 결정됩니다.  적분을 이용하여 부피와 겉넓이를 계산할 수 있습니다.

**회전체의 예시:**

* **원뿔:** 직각삼각형을 빗변을 축으로 회전시켜 만든 회전체입니다.
* **원기둥:** 직사각형을 한 변을 축으로 회전시켜 만든 회전체입니다.
* **구:** 원을 지름을 축으로 회전시켜 만든 회전체입니다.
* **토러스(도넛):** 원을 원의 중심을 지나지 않는 직선을 축으로 회전시켜 만든 회전체입니다.
* **회전타원체:** 타원을 장축 또는 단축을 축으로 회전시켜 만든 회전체입니다.


**회전체의 이해를 위한 추가 설명:**

* **회전축의 위치:** 모양도형과 회전축의 상대적인 위치에 따라 만들어지는 회전체의 모양이 달라집니다.  회전축이 모양도형의 내부에 있을 수도 있고, 외부에 있을 수도 있습니다.
* **적분의 활용:** 회전체의 부피와 겉넓이를 정확하게 계산하기 위해서는 적분을 사용하는 것이 일반적입니다.  구분구적법의 원리를 이용하여 무한히 작은 부피 또는 넓이의 합으로 계산합니다.


회전체는 수학, 공학, 디자인 등 다양한 분야에서 중요한 개념으로 사용됩니다.  예를 들어, 건축물의 설계, 엔지니어링 디자인, 컴퓨터 그래픽스 등에서 회전체의 원리를 활용합니다.

3) LLM + Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

templete = """
당신은 중학교 수학 개념을 설명하는 유용한 조수입니다. \
사용자가 특정 수학 개념이나 주제를 질문하면, 중학생이 이해할 수 있는 수준에서 해당 개념을 명확하고 상세하게 설명합니다. \
설명은 다음 구조를 따릅니다: 1) 개념의 정의를 간단히 제시, 2) 개념을 이해하기 위한 단계별 설명을 자세히 제공, 3) 실생활과 연관된 간단한 예시나 문제를 통해 구체적으로 보여주기, 4) 해당 개념이 중학교 교과과정 중 몇 학년에 주로 학습되는지 간단히 언급. \
불필요한 세부사항은 피하고, 중학생에게 적합한 쉬운 언어와 친근한 톤을 사용하며, 개념을 깊이 이해할 수 있도록 충분한 설명을 제공하세요. \
오직 개념 설명과 학년 정보만 포함하고, 그 외의 불필요한 내용은 추가하지 마세요.
"""

human_templete = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", templete),
    ("human", human_templete),
])

chain = chat_prompt | llm

In [ ]:
templete = """
당신은 중학교 수학 개념을 설명하는 유용한 조수입니다. \
사용자가 특정 수학 개념이나 주제를 질문하면, 중학생이 이해할 수 있는 수준에서 해당 개념을 명확하고 간결하게 설명합니다. \
설명은 다음 구조를 따릅니다: 1) 개념의 정의, 2) 단계별 설명, 3) 간단한 예시. \
불필요한 세부사항은 피하고, 중학생에게 적합한 쉬운 언어와 친근한 톤을 사용하세요. \
오직 개념 설명만 제공하고, 그 외의 불필요한 내용은 포함하지 마세요.
"""

human_templete = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", templete),
    ("human", human_templete),
])

chain = chat_prompt | llm

In [ ]:
response = chain.invoke("text : 회전체 대해 설명해줘" )
display(Markdown(response.content))

회전체는 어떤 도형을 하나의 직선을 축으로 하여 360도 회전시켰을 때 만들어지는 입체도형입니다.  

**1. 정의:**  평면도형을 직선 주위로 한 바퀴 돌려서 만들어지는 3차원 입체도형입니다.  회전하는 도형을 **모양체**, 회전축을 **회전축**이라고 합니다.

**2. 단계별 설명:**

1. **평면도형 선택:**  원, 삼각형, 사각형 등 회전시키고 싶은 평면도형을 선택합니다.
2. **회전축 선택:**  평면도형을 회전시킬 직선(축)을 정합니다.  이 축은 도형의 안쪽이나 바깥쪽에 있을 수 있습니다.
3. **회전:** 선택한 평면도형을 회전축을 중심으로 360도 완전히 회전시킵니다.
4. **회전체 생성:** 회전이 끝나면 만들어진 3차원 입체도형이 회전체입니다.

**3. 간단한 예시:**

* **원을 지름을 축으로 회전시키면 구가 됩니다.**  원이 회전하면서 공 모양의 입체도형이 만들어집니다.
* **직사각형을 한 변을 축으로 회전시키면 원기둥이 됩니다.** 직사각형이 회전하면서 원통 모양이 만들어집니다.
* **직각삼각형을 한 직각변을 축으로 회전시키면 원뿔이 됩니다.** 삼각형이 회전하면서 깔때기 모양이 만들어집니다.


회전체는 다양한 모양을 만들 수 있으며,  주변에서 쉽게 찾아볼 수 있는 많은 물건들이 회전체의 원리를 이용하여 만들어졌습니다.

4) RAG + LLM Prompt

- 벡터스토어에서 검색한 documents를 context로 prompt에 주입하여
  LLM에 Augmented Generation 수행

In [ ]:
# LLM + Retriever 결합으로 질문-응답 생성
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 체인 타입 (stuff, map_reduce, refine, map_rerank)
    retriever=vectorstore.as_retriever(
        search_type="mmr", # 검색 방식: MMR (Maximum Marginal Relevance)
        search_kwargs={"k": 3, "fetch_k": 10} # 검색 인자: k=3 (최종 반환 문서 수), fetch_k=10 (MMR을 위한 초기 후보군)
    ),
    return_source_documents=True # 원본 문서를 함께 반환할지 여부
)

query = "회전체 대해 설명해줘"
result = qa(query)
#display(Markdown(result["result"]))

retrieved_docs = result["source_documents"] # 검색된 문서들

# 검색된 문서들을 하나의 문자열로 결합 (예시)
# 실제 프로젝트에서는 문서 내용을 어떻게 조합할지 더 고민해볼 수 있어요.
docs_content = "\n\n".join([doc.page_content for doc in retrieved_docs])

#formatted_messages = chat_prompt.format_messages(context=docs_content, text=query)
#response = llm.invoke(formatted_messages)

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": lambda x: docs_content, "text": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

#response = rag_chain.invoke("선생님, 순환소수란 정확히 뭐예요? 그냥 무한소수랑 다른 점이 뭔가요?")
response = rag_chain.invoke(query)

display(Markdown(response))

회전체는 어떤 도형을 **직선을 축으로 하여 회전시켰을 때 만들어지는 3차원 입체 도형**입니다.  쉽게 말해, 평면 도형을 빙글빙글 돌려서 만든 입체 모양이라고 생각하면 됩니다.

**단계별 설명:**

1. **평면 도형 선택:**  원, 삼각형, 사각형 등 어떤 평면 도형을 선택합니다.
2. **회전축 선택:**  도형을 회전시킬 직선을 선택합니다. 이 직선은 도형의 일부분을 지나거나, 도형 밖에 있을 수도 있습니다.
3. **회전:** 선택한 도형을 회전축을 중심으로 360도 회전시킵니다.
4. **회전체 생성:** 회전 과정에서 만들어지는 3차원 입체 도형이 회전체입니다.


**간단한 예시:**

* **원을 지름을 축으로 회전시키면 구가 됩니다.**  (공처럼 생긴 모양)
* **직사각형을 한 변을 축으로 회전시키면 원기둥이 됩니다.** (캔이나 컵처럼 생긴 모양)
* **직각삼각형을 한 직각변을 축으로 회전시키면 원뿔이 됩니다.** (아이스크림 콘처럼 생긴 모양)


회전축의 위치와 도형의 모양에 따라 다양한 회전체가 만들어집니다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# LLM + Retriever 결합으로 질문-응답 생성
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 체인 타입 (stuff, map_reduce, refine, map_rerank)
    retriever=vectorstore.as_retriever(
        search_type="mmr", # 검색 방식: MMR (Maximum Marginal Relevance)
        search_kwargs={"k": 3, "fetch_k": 10} # 검색 인자: k=3 (최종 반환 문서 수), fetch_k=10 (MMR을 위한 초기 후보군)
    ),
    return_source_documents=True # 원본 문서를 함께 반환할지 여부
)

rag_chain = (
    {"context": lambda x: docs_content, "text": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

def get_response_with_docs(question: str):
    result = qa(question)
    docs = result["source_documents"]

    for i, doc in enumerate(docs):
      print(f"\n--- Document {i+1} ---")
      print(doc.page_content[:1000])  # 처음 1000자만 출력

    context = "\n\n".join([doc.page_content for doc in docs])
    return rag_chain.invoke({"text": question})

# 질문 예시
#print(get_response_with_docs("소인수분해는 어떻게 하나요?"))
#print(get_response_with_docs("순환소수와 무한소수의 차이는?"))

In [ ]:
display(Markdown(get_response_with_docs("소인수분해는 어떻게 하나요?")))


--- Document 1 ---
학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 어떻게 되나요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다.

--- Document 2 ---
학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 어떻게 되나요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다.

--- Document 3 ---
학습 목표: [9수04-07] 다면체의 성질을 이해한다.

질문: 무엇에 대해 말하나요?

답변: 칠면체에 대해 말합니다. 

설명: 칠면체이다.


소인수분해는 어떤 수를 소수들의 곱으로 나타내는 것을 말해요.  소수는 1과 자기 자신만으로 나누어 떨어지는 1보다 큰 수를 말하죠. (예: 2, 3, 5, 7, 11, ...)

소인수분해 하는 방법은 다음과 같아요:

1. **주어진 수를 가장 작은 소수로 나눕니다.**  나누어 떨어지지 않으면 다음 작은 소수로 나누어 봅니다.
2. **몫을 다시 가장 작은 소수로 나눕니다.**  1이 될 때까지 이 과정을 반복합니다.
3. **나눈 소수들을 모두 곱하면 원래 수가 됩니다.** 이 소수들이 바로 그 수의 소인수들이에요.

**예시:** 24를 소인수분해 해 볼까요?

1. 24는 2로 나누어떨어지므로, 24 ÷ 2 = 12
2. 12도 2로 나누어떨어지므로, 12 ÷ 2 = 6
3. 6도 2로 나누어떨어지므로, 6 ÷ 2 = 3
4. 3은 소수이므로 더 이상 나눌 수 없어요.

따라서 24의 소인수분해는 2 × 2 × 2 × 3 = 2³ × 3 이 됩니다.  2와 3이 24의 소인수이고, 2는 세 번 곱해졌으므로 지수는 3이 됩니다.

In [ ]:
display(Markdown(get_response_with_docs("순환소수와 무한소수의 차이는?")))


--- Document 1 ---
학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 어떻게 되나요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다.

--- Document 2 ---
학습 목표: [9수04-08] 회전체의 성질을 이해한다.

질문:  단면의 모양이 어떻게 되나요?

답변: 단면의 모양은 원입니다.

설명: (2) 단면의 모양은 원이다.

--- Document 3 ---
학습 목표: [9수04-07] 다면체의 성질을 이해한다.

질문: 무엇에 대해 말하나요?

답변: 칠면체에 대해 말합니다. 

설명: 칠면체이다.


순환소수와 무한소수는 모두 소수점 아래 숫자가 무한히 이어지는 소수이지만,  숫자의 배열 패턴에 차이가 있습니다.

**1. 정의:**

* **무한소수:** 소수점 아래 숫자가 무한히 계속되는 소수입니다.  순환하는 부분이 없을 수도 있고, 있을 수도 있습니다.

* **순환소수:** 소수점 아래 어떤 숫자 또는 숫자들의 배열이 무한히 반복되는 무한소수입니다.  반복되는 부분을 '순환마디'라고 합니다.


**2. 단계별 설명:**

무한소수는 크게 두 가지로 나눌 수 있습니다.

* **순환소수:**  예를 들어 0.3333... (3이 무한히 반복)은 순환소수입니다.  순환마디는 '3'입니다.  또, 0.123123123... (123이 무한히 반복)도 순환소수이며, 순환마디는 '123'입니다.  순환마디는 윗점(.)이나 괄호()를 사용하여 표시합니다 (예: 0.3̇ 또는 0.(3), 0.123̇ 또는 0.(123)).

* **비순환소수:**  소수점 아래 숫자가 무한히 계속되지만, 어떤 규칙적인 패턴도 반복되지 않는 무한소수입니다. 예를 들어 원주율 π(파이) = 3.1415926535... 는 비순환소수입니다.  숫자들이 계속 이어지지만,  규칙적인 반복 패턴은 없습니다.


**3. 간단한 예시:**

* **순환소수:** 1/3 = 0.3333... = 0.3̇  (순환마디는 3)
* **비순환소수:**  √2 (루트 2) ≈ 1.41421356... (규칙적인 반복 패턴이 없음)


결론적으로, 모든 순환소수는 무한소수이지만, 모든 무한소수가 순환소수인 것은 아닙니다.  순환소수는 규칙적인 반복 패턴을 가지는 무한소수인 반면, 비순환소수는 그렇지 않습니다.

# < 회고 >

## Vector DB에서 RetreivalQA의 결과가 제대로 나오지 않는 문제가 보인다. 질의가 뭐든지, 비슷한 문서를 검색하는 양상이다.
 - 확인한 사항
  + docs에 포함되어 있는지? yes
  + vector에 해당 데이타(문자)가 포함되어 있는지? yes
  + 가져올 문서 개수를 증가 3 -> 5 -> 10
  + mmr -> similarity 로 변경
   